# <u>Data Preprocessing using Python

## Background
You have been given a collection of data from a company wishing to process its customer
records for business purposes (acw_user_data.csv). The existing systems in-place at the
company only export to a CSV file, and this is not in an appropriate format for analysis.
1. __You
have been given the task of preparing this data for further analyses by your colleagues
within the company, including representation changes, filtering, and deriving some new
attributes / metrics for them.
These data include attributes such as first name, second name, credit card number, marital
status, and even contains data on the customer’s car.__
2. __The number of records provided is significant, and therefore it is expected that solutions are
robust to varying types of data, and varying values, offering a programmatic solution.__

## Data Processing
__In this section, seven tasks will be carried out relating to pre-processing data, and only standard python (No pandas / seaborn) with default libraries (os, sys, time, json, csv, etc.) will be used to carry out these tasks__

__Importing the relevant default libraries to be used for the tasks__

In [1]:
#Importing the relevant libraries for the tasks.
#only default Python libraries will be used

import os
import sys
import csv 
import json           
import datetime
from datetime import datetime
from datetime import timedelta
from datetime import date

__Creating Classes which will serve as a container to hold the customer data__
- The classes have been created after having taken a look at the structure of the acw_user_data csv file

In [2]:
#creating a parent class of customers which will contain attributes for each customer
#The idea here is that each customer will have the different attributes contained in the inner classes

class Customer:
    
    #using the init special method and constructor which is to be called when when the class is being initiated
    #this will define the characteristics of each customer
    def __init__(self):
        self.personal = self.Personal()
        self.address = self.Address()
        self.work = self.Work()
        self.card = self.Card()
        self.vehicle = self.Vehicle()

    #define the repr method for the main class Customer    
    def __repr__(self):
        return f'[{self.Personal}, {self.Address},{self.Work},{self.Card},{self.Vehicle}]' 
    
    #define the inner class Personal for the Parent Class Customer and its attributes
    class Personal():
        def __init__(self, first_name, last_name, sex, age, marital_status, dependants):
        #a customer's Personal data must contain these six variables
            self.first_name = first_name
            self.last_name = last_name
            self.sex = sex
            self.age = age
            self.marital_status = marital_status
            self.dependants = dependants
            
        #define the repr method for the class Personal
        def __repr__(self):
            return f'[{self.first_name}, {self.last_name},{self.sex},{self.age},{self.marital_status},{self.dependants}]'
    
    #define the inner class Address for the Parent Class Customer and its attributes
    class Address:
        def __init__(self, street, city, postcode):
        #a customer's Address data must contain these three variables    
            self.street = street
            self.city = city
            self.postcode = postcode
            
        #define the repr method for the class Address
        def __repr__(self):
            return f'[{self.street}, {self.city},{self.postcode}]'
    
    #define the inner class Work for the Parent Class Customer and its attributes
    class Work:
        def __init__(self, employer, commute, salary_GBP, retired, pension_GBP):
        #a customer's Work data must contain these five variables 
            self.employer = employer
            self.commute = commute
            self.salary_GBP = salary_GBP
            self.retired = retired
            self.pension_GBP = pension_GBP

        #define the repr method for the class Work
        def __repr__(self):
            return f'[{self.employer}, {self.commute},{self.salary_GBP},{self.retired},{self.pension_GBP}]'            

    #define the inner class Card for the Parent Class Customer and its attributes        
    class Card:
        def __init__(self, card_num, card_cvv, card_iban, card_start, card_exp):
        #a customer's Card data must contain these five variables     
            self.card_num = card_num
            self.card_cvv = card_cvv
            self.card_iban = card_iban
            self.card_start = card_start
            self.card_exp = card_exp
            
        #define the repr method for the class Card
        def __repr__(self):
            return f'[{self.card_num}, {self.card_cvv},{self.card_iban},{self.card_start},{self.card_exp}]'

    #define the inner class Vehicle for the Parent Class Customer and its attributes       
    class Vehicle:
        def __init__(self, vehicle_make, vehicle_model, vehicle_year, vehicle_type):
        #a customer's Vehicle data must contain these four variables  
            self.vehicle_make = vehicle_make
            self.vehicle_model = vehicle_model
            self.vehicle_year = vehicle_year
            self.vehicle_type = vehicle_type

        #define the repr method for the class Vehicle
        def __repr__(self):
            return f'[{self.vehicle_make}, {self.vehicle_model},{self.vehicle_year},{self.vehicle_type}]'  

__Tasks 1-3: In the next cell, the following will be carried out:__
1. Reading in the csv file
2. Converting values to their appropiate types
3. Converting the flat csv structure to a nested structure

__A function will be defined to hold all of these points__
<br>__Note regarding type conversion:__
- Age is handled and converted to int
- Dependants is handled - converted to int and then rows with issues handled using try/execpt conditions
- Commute data is handled - converted to float
- Salary is handled - converted to int
- Pension is handled - converted to int
- Card number is handled - using a combination of int and float due to the numbers ending with several zeros
- Card start date and end date is handled - converted to datetime and back to string for ease of conversion to json later
- Card cvv is handled - converted to int
- Retired is handled to be a boolean - converted from string
- Vehicle year is handled - converted to int

In [3]:
#creating a function which takes a filename as the input
#the function will do the required type conversions and create the nested structure

def customer(file_name):
    
    #the list comprehensions required for the tasks
    cust_data = []
    problematic_rows = []
      
    try:
        #reading in the csv file
        with open (file_name,'r') as cust_file:
            csv_reader = csv.reader(cust_file, delimiter=',')
            headers = next(csv_reader)
            for row_no , row in enumerate(csv_reader):
                
                #the type conversions are done first
                #in this case, the card start and end dates needed extensive conversion efforts
                #this is because the date styles when taken from csv to Python are not consistent
                #in most rows there were mainly 'Month-Year', while in some rows it was 'Year-Month'
                #these needed to be addressed
                
                try:
                    #direct datetime conversion and put the data in strings
                    row[6] = str(datetime.strptime(row[6], '%b-%y').date())
                    row[7] = str(datetime.strptime(row[7], '%b-%y').date())

                except Exception as e:
                    try:
                        #the idea here is to split the odd rows with 'Year-Month' instead of 'Month-Year' into two
                        #then after splitting, join them back in the row order and then do the datetime conversion
                        
                        cc_start_split = row[6].split('-')
                        cc_start_join = cc_start_split[1] + '-' + cc_start_split[0]
                        row[6] = str(datetime.strptime(cc_start_join, '%b-%y').date())

                        row[7] = str(datetime.strptime(row[7], '%b-%y').date())

                    except Exception as e:
                        try:
                            cc_exp_split = row[7].split('-')
                            cc_exp_join = cc_exp_split[1] + '-' + cc_exp_split[0]
                            row[7] = str(datetime.strptime(cc_exp_join, '%b-%y').date())

                        except Exception as e:
                            print ('datetime error')
                
                #another type conversion is done here to change TRUE and FALSE to boolean
                if row[16] == 'TRUE':
                    row[16] = True
                elif row[16] == 'FALSE':
                    row[16] = False
                else:
                    row[16] ='N/A'
                
                
                #finally, after the tricky date conversions, data storage is done with some other simple type conversions
                #the Parent class and inner classes have been assigned attributes as noted in the previous cells
                #each of the class takes the required attributes otherwise an error is raised
                
                try:
                    personal_data = Customer.Personal(row[11],row[13],row[18],int(row[3]), row[14], int(row[10])).__dict__
                    address_data = Customer.Address(row[0],row[1],row[2]).__dict__
                    work_data = Customer.Work(row[5],float(row[4]),int(row[17]),row[16],int(row[15])).__dict__
                    card_data = Customer.Card(int(float(row[8])),int(row[9]),row[12],row[6],row[7]).__dict__
                    vehicle_data = Customer.Vehicle(row[19],row[20],int(row[21]),row[22]).__dict__
                    
                    #the list comprehension defined earlier is apppended and now holds the data
                    cust_data.append({
                        'personal':personal_data,
                        'address': address_data,
                        'work': work_data,
                        'card': card_data,
                        'vehicle':vehicle_data,
                                         })
                except Exception as err:
                    
                    #converting dependants raises errors and is dealt with here
                    problematic_rows.append(row_no)
                    
                    #where there are blanck cells, change this to 0
                    if row[10] == "":
                        row[10] = 0
                        
                        personal_data = Customer.Personal(row[11],row[13],row[18],int(row[3]), row[14], int(row[10])).__dict__
                        address_data = Customer.Address(row[0],row[1],row[2]).__dict__
                        work_data = Customer.Work(row[5],float(row[4]),int(row[17]),row[16],int(row[15])).__dict__
                        card_data = Customer.Card(int(float(row[8])),int(row[9]),row[12],row[6],row[7]).__dict__
                        vehicle_data = Customer.Vehicle(row[19],row[20],row[21],row[22]).__dict__

                        cust_data.append({
                            'personal':personal_data,
                            'address': address_data,
                            'work': work_data,
                            'card': card_data,
                            'vehicle':vehicle_data,
                                             })

            print('There are', (len(problematic_rows)), 'rows with conversion issues and the problematic rows are:\n', problematic_rows)
            print ('The problematic rows had blank data and this has been changed to 0')
            print('\n The length of the list is:', len(cust_data),'rows')
                
        return (cust_data)
    
    except FileNotFoundError:
         print('CSV file name in the with open ()line is wrong\nPlease check the file name correctly or(and) check the directory')        

#creating an object
#calling the function and using the filename as the input to return our results
all_customers = customer('acw_user_data.csv')
all_customers

There are 19 rows with conversion issues and the problematic rows are:
 [21, 109, 179, 205, 270, 272, 274, 358, 460, 468, 579, 636, 679, 725, 822, 865, 917, 931, 983]
The problematic rows had blank data and this has been changed to 0

 The length of the list is: 1000 rows


[{'personal': {'first_name': 'Kieran',
   'last_name': 'Wilson',
   'sex': 'Male',
   'age': 89,
   'marital_status': 'married or civil partner',
   'dependants': 3},
  'address': {'street': '70 Lydia isle',
   'city': 'Lake Conor',
   'postcode': 'S71 7XZ'},
  'work': {'employer': 'N/A',
   'commute': 0.0,
   'salary_GBP': 72838,
   'retired': True,
   'pension_GBP': 7257},
  'card': {'card_num': 676000000000,
   'card_cvv': 875,
   'card_iban': 'GB62PQKB71416034141571',
   'card_start': '2018-08-01',
   'card_exp': '2027-11-01'},
  'vehicle': {'vehicle_make': 'Hyundai',
   'vehicle_model': 'Bonneville',
   'vehicle_year': 2009,
   'vehicle_type': 'Pickup'}},
 {'personal': {'first_name': 'Jonathan',
   'last_name': 'Thomas',
   'sex': 'Male',
   'age': 46,
   'marital_status': 'married or civil partner',
   'dependants': 1},
  'address': {'street': '00 Wheeler wells',
   'city': 'Chapmanton',
   'postcode': 'L2 7BT'},
  'work': {'employer': 'Begum-Williams',
   'commute': 13.72,
   's

### Task 4: Converting our new list of dict to json format - a list of dictionaries, where each index of the list is a dictionary representing a singular person.

__A function that will be used to write json files will be created.__
- This function will take two arguments: the name of the list to be written and the name of the new file to be created

In [4]:
#creating the function
#the function takes 2 arguments
def create_json(list_name,file_name):
    with open(file_name, mode='w') as processed_json:
        json.dump(list_name, processed_json)

#creating the object for the new filename to be created
#calling the function, which then creates the new file.
#in this case, the new file is called processed.
processed_json = 'processed.json'
create_json(all_customers,processed_json)

### Task 5: You should create two additional file outputs, retired.json and employed.json, these should contain all retired customers (as indicated by the retired field in the CSV), and all employed customers respectively (as indicated by the employer field in the CSV) and be in the JSON data format. 

In [5]:
#creating the list comprehensions for the new lists
retired_customers = []
employed_customers = []
not_retired_employed = [] #created to handle exceptions

#iterating through the all_customers list to separate the new lists we want
#This iteration is done based on set conditions
#if row is retired = TRUE then the customer is retired
#if row employed is not equal to N/A, then the customer is employed

for row_no, row in enumerate (all_customers):
    if row['work'] ['retired'] == True:
        retired_customers.append(row)
    elif row['work'] ['employer'] != 'N/A':
        employed_customers.append(row)
    else:
        not_retired_employed.append(row) #created to handle exceptions

#creating the objects for the filename to be used for creating the json files
retired_json = 'retired.json'
employed_json = 'employed.json'

#calling the previously created function for creating json files
#the inputs are the list (in this case retired and employed) and the objects for the file names
create_json(retired_customers,retired_json)
create_json(employed_customers,employed_json)

print('The number of retired customers are:',len(retired_customers))
print('The number of employed customers are:',len(employed_customers))

The number of retired customers are: 246
The number of employed customers are: 754


In [6]:
retired_customers

[{'personal': {'first_name': 'Kieran',
   'last_name': 'Wilson',
   'sex': 'Male',
   'age': 89,
   'marital_status': 'married or civil partner',
   'dependants': 3},
  'address': {'street': '70 Lydia isle',
   'city': 'Lake Conor',
   'postcode': 'S71 7XZ'},
  'work': {'employer': 'N/A',
   'commute': 0.0,
   'salary_GBP': 72838,
   'retired': True,
   'pension_GBP': 7257},
  'card': {'card_num': 676000000000,
   'card_cvv': 875,
   'card_iban': 'GB62PQKB71416034141571',
   'card_start': '2018-08-01',
   'card_exp': '2027-11-01'},
  'vehicle': {'vehicle_make': 'Hyundai',
   'vehicle_model': 'Bonneville',
   'vehicle_year': 2009,
   'vehicle_type': 'Pickup'}},
 {'personal': {'first_name': 'Clive',
   'last_name': 'Evans',
   'sex': 'Male',
   'age': 67,
   'marital_status': 'single',
   'dependants': 1},
  'address': {'street': '81 Goodwin dam',
   'city': 'Griffinstad',
   'postcode': 'G3 7ZX'},
  'work': {'employer': 'N/A',
   'commute': 0.0,
   'salary_GBP': 27964,
   'retired': Tru

In [7]:
employed_customers

[{'personal': {'first_name': 'Jonathan',
   'last_name': 'Thomas',
   'sex': 'Male',
   'age': 46,
   'marital_status': 'married or civil partner',
   'dependants': 1},
  'address': {'street': '00 Wheeler wells',
   'city': 'Chapmanton',
   'postcode': 'L2 7BT'},
  'work': {'employer': 'Begum-Williams',
   'commute': 13.72,
   'salary_GBP': 54016,
   'retired': False,
   'pension_GBP': 0},
  'card': {'card_num': 4530000000000000,
   'card_cvv': 583,
   'card_iban': 'GB37UMCO54540228728019',
   'card_start': '2008-12-01',
   'card_exp': '2026-11-01'},
  'vehicle': {'vehicle_make': 'Nissan',
   'vehicle_model': 'ATS',
   'vehicle_year': 1996,
   'vehicle_type': 'Coupe'}},
 {'personal': {'first_name': 'Antony',
   'last_name': 'Jones',
   'sex': 'Male',
   'age': 22,
   'marital_status': 'married or civil partner',
   'dependants': 1},
  'address': {'street': 'Studio 33K Joel walk',
   'city': 'Randallborough',
   'postcode': 'ME3N 1GH'},
  'work': {'employer': 'Hill-Wright',
   'commute'

### Task 6: there may be some issues with credit card entries. Any customers that have more than 10 years between their start and end date need writing to a separate file, called remove_ccard.json, in the JSON data format. The client will manually deal with these later based on your output. They request that you write a function to help perform this, which accepts a single row from the CSV data.

In [8]:
#first, a function is create which accepts a row
#the string dates converted earlier are converted back to datetime.date using the datetime function
#after this timedelta is applied to show the number of days that make up 10 years

def card_issues(row):
    flagged = False
    
    #the card rows
    cc_start = row['card']['card_start']
    cc_exp = row['card']['card_exp']
    
    #converting the string dates back to time
    cc_start_dt = datetime.strptime(cc_start, '%Y-%m-%d').date()
    cc_exp_dt = datetime.strptime(cc_exp, '%Y-%m-%d').date()
    
    #creating an object
    #this is the difference of card expiry - card start date
    #due to the datetime applied, results are returned in days
    
    diff_days = cc_exp_dt - cc_start_dt
    
    #the conditional statement to enforce the results for the function
    if diff_days > timedelta(days=3652):
        return True
    else:
        return flagged

__After the function is created, testing is done below to see if it indeeds picks up the correct rows. The first 10 rows has been checked below and results were compared with a check done manually__

In [9]:
(card_issues(all_customers[0]), 
 card_issues(all_customers[1]),
 card_issues(all_customers[2]), 
 card_issues(all_customers[3]), 
 card_issues(all_customers[4]),
 card_issues(all_customers[5]), 
 card_issues(all_customers[6]),
 card_issues(all_customers[7]), 
 card_issues(all_customers[8]),
 card_issues(all_customers[9]))

(False, True, False, False, False, False, True, True, False, False)

__Lastly, a list is created and a json file is created from this list to store this data__

In [10]:
#creating the list comprehension to hold the remove_ccard data
#iterating through the all_customers list, then applying the card_issues function to get the rows
#these rows are then appended to the list comprehension

remove_ccard = []
for row in all_customers:
    if card_issues(row) == True:
        remove_ccard.append(row)

#the function for creating json files created earlier is applied again
#the object for the file name is created as well
remove_ccard_json = 'remove_ccard.json'
create_json(remove_ccard,remove_ccard_json)
print ('The number of customers where the difference between card start and expiry dates is more than 3652 days :',len(remove_ccard))

The number of customers where the difference between card start and expiry dates is more than 3652 days : 283


### Task 7: You have been tasked with calculating some additional metrics which will be used for ranking customers. You should create a new data attribute for our customers called “Salary-Commute”. Reading in from processed.json:
1. __Add, and calculate appropriately, this new attribute. It should represent the Salary that a customer earns, per mile of their commute.__
  <br>- Note: If a person travels 1 or fewer commute miles, then their salarycommute would be just their salary.
2. __Sort these records by that new metric, in ascending order.__
3. __Store the output file out as a JSON format, for a commute.json file.__


In [11]:
#creating a function whose input is the filename to be processed

def salary_commute(json_file):
    salary_per_commute = []
    
    try:
        with open(json_file, mode='r') as processed_file:
            new_metric = json.load(processed_file)
            for row in new_metric:
                if (row['work']['commute'] <= 1):
                    row['salary_commute'] = row['work']['salary_GBP']
            
                else:
                    row['salary_commute'] = row['work']['salary_GBP'] / row['work']['commute']
                salary_per_commute.append(row)
        
    except Exception as err:
        print('There is an error')
    
    sorted_salary_commute = sorted(salary_per_commute, key=lambda d: d['salary_commute']) 
    return (sorted_salary_commute)

#calling the function
salary_commute_obj = salary_commute('processed.json')

#again using the function created earlier to convert to json
#creating the object for the json filename
commute_json = 'commute.json'
create_json(salary_commute_obj,commute_json)

salary_commute_obj

[{'personal': {'first_name': 'Graeme',
   'last_name': 'Jackson',
   'sex': 'Male',
   'age': 52,
   'marital_status': 'single',
   'dependants': 2},
  'address': {'street': 'Studio 9 Reid lights',
   'city': 'South Ryan',
   'postcode': 'E27 9GY'},
  'work': {'employer': 'Smith, Birch and Burke',
   'commute': 5.52,
   'salary_GBP': 17046,
   'retired': False,
   'pension_GBP': 0},
  'card': {'card_num': 4710000000000000,
   'card_cvv': 3053,
   'card_iban': 'GB09ELJH35362236053720',
   'card_start': '2014-06-01',
   'card_exp': '2029-04-01'},
  'vehicle': {'vehicle_make': 'Chevrolet',
   'vehicle_model': 'Rally Wagon 1500',
   'vehicle_year': 2011,
   'vehicle_type': 'SUV'},
  'salary_commute': 3088.04347826087},
 {'personal': {'first_name': 'Janet',
   'last_name': 'Quinn',
   'sex': 'Female',
   'age': 30,
   'marital_status': 'married or civil partner',
   'dependants': 1},
  'address': {'street': 'Studio 34r Wilkinson camp',
   'city': 'Louisland',
   'postcode': 'G7H 8FA'},
  'w